### Instalação dos pacotes necessários

Pandas: Biblioteca para manipulação e análise de dados
PyArrow: Biblioteca para leitura e escrita de arquivos parquet
Spotipy: Biblioteca para acesso a API do Spotify

Use o comando abaixo para instalar os pacotes necessários:
```bash
$ pip install pandas pyarrow spotipy
```

Ou instale as dependências diretamente do arquivo `requirements.txt`:
```bash
$ pip install -r requirements.txt
```

### Configuração da API do Spotify com um .env

Para acessar a API do Spotify é necessário criar um aplicativo no [Spotify for Developers](https://developer.spotify.com/dashboard/applications) e obter as credenciais de acesso.

Crie um arquivo `.env` na raiz do projeto e adicione as seguintes variáveis de ambiente:
```bash
SPOTIPY_CLIENT_ID=seu_client_id
SPOTIPY_CLIENT_SECRET=sua_client_secret
```

In [67]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import glob

In [68]:
env_vars = {}

# Le o arquivo .env
with open('.env') as f:
    env_vars = dict(
        tuple(line.replace('"', '').replace("'", '').strip().split('=', 1)) for line in f
    )

# Cria o objeto de autenticação
client_credentials_manager = SpotifyClientCredentials(
    client_id=env_vars['SPOTIPY_CLIENT_ID'],
    client_secret=env_vars['SPOTIPY_CLIENT_SECRET']
)

# Configurando as credenciais do cliente
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [69]:
# Função para obter as 100 playlists mais populares de música independente do Brasil
def get_playlists(query = '', directory = '', n = 1, offset = 50, market = 'BR', verbose = False):
    all_tracks = 0
    all_playlists = []
    for i in range(n):
        try:
            playlists = sp.search(query, limit=offset, offset=i*offset, type='playlist', market = market)
            # Printando o nome das playlists
            for item in playlists['playlists']['items']:
                play_tracks = 0
                if verbose:
                    print(f"Playlist encontrada: {item['name']}")
                all_playlists.append(item)
                # Obtendo as tracks da playlist
                play_id = item['id']
                if verbose:
                    print(f"Obtendo músicas da playlist: {play_id}")
                tracks = get_playlist_tracks(play_id, market = market)
                # Salvando o objeto de tracks em um arquivo parquet
                df = pd.DataFrame(tracks)
                # Convertendo o DataFrame para um Arrow Table
                table = pa.Table.from_pandas(df)
                # Salvando o arquivo parquet
                pq.write_table(table, directory + f'/tracks_{play_id}.parquet')
                # Incrementando o contador de músicas
                play_tracks += len(tracks)
                if verbose:
                    print(f"Músicas encontradas: {play_tracks}")
                    print("-------------------------------------------------------")
                all_tracks += play_tracks
        except Exception as e:
            print("Erro ao obter playlists: ", e)
            continue

    # Salvando o objeto de playlists em um arquivo parquet
    df = pd.DataFrame(all_playlists)
    # Convertendo o DataFrame para um Arrow Table
    table = pa.Table.from_pandas(df)
    # Salvando o arquivo parquet
    pq.write_table(table, directory + '/playlists.parquet')
    if verbose:
        print(f"Playlists encontradas: {len(all_playlists)}")
        print(f"Musicas encontradas: {all_tracks}")
    return all_playlists

# Função para obter todas as músicas de uma playlist
def get_playlist_tracks(playlist_id, limit = 100, market = 'BR'):
    all_tracks = []
    try:
        results = sp.playlist_tracks(playlist_id, limit = limit, market = market)
        while results['next']:
            results = sp.next(results)
            all_tracks.extend(results['items'])
    except Exception as e:
        print("Erro ao obter músicas da playlist: ", e)
    return all_tracks


In [70]:
# Unificar todos os arquivos parquet de tracks do diretório
def unify_track_files(directory):

    # Lista todos os arquivos parquet do diretório
    files = glob.glob(directory + '/tracks_*.parquet')

    for file in files:
        filename = file.split('/')[-1].replace('.parquet', '')

        table = pq.read_table(file)
        df = table.to_pandas()

        new_df = pd.DataFrame()

        for row in df.iterrows():
            if not row[1]['track']:
                continue
            # Converte cada linha em uma entrada na tabela com colunas
            new_row = {
                'id': row[1]['track']['id'] if row[1]['track']['id'] else None,
                'nome': row[1]['track']['name'] if row[1]['track']['name'] else None,
                'popularidade': row[1]['track']['popularity'] if row[1]['track']['popularity'] else None,
                'track': row[1]['track']['track'] if row[1]['track']['track'] else None,
                'num_track': row[1]['track']['track_number'] if row[1]['track']['track_number'] else None,
                'tipo': row[1]['track']['type'] if row[1]['track']['type'] else None,
                'album': row[1]['track']['album']['name'] if row[1]['track']['album']['name'] else None,
                'artistas': [{'nome': artist['name'], 'id': artist['id']}
                    if artist and artist['name'] and artist['id'] else None
                    for artist in row[1]['track']['artists']],
                'num_disc': row[1]['track']['disc_number'] if row[1]['track']['disc_number'] else None,
                'duracao': row[1]['track']['duration_ms'] if row[1]['track']['duration_ms'] else None,
                'explicita': row[1]['track']['explicit'] if row[1]['track']['explicit'] else None,
                'ids_externos': row[1]['track']['external_ids'] if row[1]['track']['external_ids'] else None,
                'urls_externas': row[1]['track']['external_urls'] if row[1]['track']['external_urls'] else None,
                'href': row[1]['track']['href'] if row[1]['track']['href'] else None,
                'local': row[1]['track']['is_local'] if row[1]['track']['is_local'] else None,
                'tocavel': row[1]['track']['is_playable'] if row[1]['track']['is_playable'] else None,
                'preview_url': row[1]['track']['preview_url'] if row[1]['track']['preview_url'] else None,
                'uri': row[1]['track']['uri'] if row[1]['track']['uri'] else None,
            }
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

        # Exportando o DataFrame para um novo arquivo parquet
        table = pa.Table.from_pandas(new_df)
        pq.write_table(table, directory + '/pre_proc_' + filename + '.parquet')

In [71]:
def unify_playlist_files(directory):

    # Lista todos os arquivos parquet do diretório
    files = glob.glob(directory + '/playlists.parquet')

    for file in files:
        filename = file.split('/')[-1].replace('.parquet', '')

        table = pq.read_table(file)
        df = table.to_pandas()

        new_df = pd.DataFrame()

        for row in df.iterrows():
            # Converte cada linha em uma entrada na tabela com colunas
            # Nome da Playlist, Descrição, Link Externo, Proprietário, Total de Faixas
            new_row = {
                'id': row[1]['id'],
                'nome': row[1]['name'],
                'descricao': row[1]['description'],
                'link': row[1]['external_urls']['spotify'],
                'proprietario': row[1]['owner']['display_name'],
                'n_faixas': row[1]['tracks']['total']
            }
            new_df = pd.concat([new_df, pd.DataFrame([new_row])], ignore_index=True)

        # Exportando o DataFrame para um novo arquivo parquet
        table = pa.Table.from_pandas(new_df)
        pq.write_table(table, directory + '/pre_proc_' + filename + '.parquet')

### Execução do script

Usando a lista de queries fornecida, o script irá buscar playlists no Spotify e salvar os dados em um arquivo parquet
relacionado a cada query.

Além disso, para cada playlist encontrada, o script irá buscar as músicas e salvar os dados em um arquivo parquet
relacionado a cada playlist.

In [72]:
# Lista de queries para buscar playlists
# Essa lista vai definir quais playlists serão buscadas
# e vai dividir as rodadas em pastas diferentes
query_list = [
    'independente brasil',
    'indie carioca',
    'brasil indie'
]

for query in query_list:
    # Criar a pasta para a query
    directory = 'queries/' + query.replace(' ', '_')
    if not os.path.exists(directory):
        os.makedirs(directory)

    print(f"Buscando playlists para a query: {query}")
    playlists = get_playlists(query, directory = directory, n = 50, offset = 50, market = 'BR', verbose = False)
    print(f"Playlists encontradas: {len(playlists)}")

    # Unificar os arquivos parquet de tracks
    unify_track_files(query)
    unify_playlist_files(query)

Buscando playlists para a query: independente brasil
Playlists encontradas: 5
Buscando playlists para a query: indie carioca
Playlists encontradas: 5
Buscando playlists para a query: brasil indie
Playlists encontradas: 5
